In [1]:
from peft import *
from sentence_transformers  import SentenceTransformer,CrossEncoder
from sentence_transformers.util import semantic_search
import torch
import json
import os
from bs4 import BeautifulSoup,PageElement,Comment,NavigableString
import pandas as pd
cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'
def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)

No ROCm runtime is found, using ROCM_HOME='/usr'


In [2]:
if True:
    model_name = 'cross-encoder/msmarco-MiniLM-L12-en-de-v1'
    model = CrossEncoder(model_name,num_labels=1,cache_dir=cache_dir,device='cuda')
    model
else:
    model = CrossEncoder(output_dir+'click')
    model.max_length=512

In [3]:
model.max_length=512

In [4]:
mypd = pd.read_excel('click_train_data_new_truncated_2.xlsx',index_col=0)
mypd


,outerHTML,filename,nid,negatives,negatives_query,query,element,history,history_query
0,"<a xml:lang=""en-US"" lang=""en-US"" class=""btn bt...",2024-07-27T06_52_57_411Z.html,434,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""language"",""2"":""sign in"",""3"":""empl...",apply now,"<a class=""btn btn-primary btn-large btn-lg app...",{},{}
1,"<input type=""text"" name=""username"" id=""usernam...",2024-07-27T06_53_08_168Z.html,489,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""5""...",email address,"<input aria-describedby=""signIn_EmailError"" ar...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now""}"
2,"<input type=""password"" name=""password"" id=""pas...",2024-07-27T06_53_10_623Z.html,497,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",password,"<input aria-describedby=""signIn_PasswordError""...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now"",""1"":""email address""}"
3,"<button onclick=""return validateFields();"" nid...",2024-07-27T06_53_12_219Z.html,505,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",sign in register,"<button clickable=""true"" nid=""505"" onclick=""re...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now"",""1"":""email address"",""2"":""pass..."
4,"<a href=""/career?company=Koerber&amp;site=VjIt...",2024-07-27T06_53_21_295Z.html,523,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",create an account,"<a clickable=""true"" href=""/career?company=Koer...","{""1"":""<input type=\""text\"" name=\""username\"" i...","{""1"":""email address"",""2"":""password"",""3"":""sign ..."
...,...,...,...,...,...,...,...,...,...
722,"<div class=""profileResumeContentStyle-398"" nid...",2024-07-27T14_35_33_533Z.html,458,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":""data privacy notice"",""1"":""thirdparty coo...",add resumeupload a file or generate a resume u...,"<div class=""profileResumeContentStyle-398"" cli...","{""0"":""<i data-icon-name=\""CheckMark\"" aria-hid...","{""0"":""optional"",""1"":""as a part of the online a..."
723,"<input type=""text"" id=""TextField36"" aria-requi...",2024-07-27T14_35_53_819Z.html,486,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":""data privacy notice"",""1"":""thirdparty coo...",most recent employer,"<input aria-describedby=""TextFieldDescription3...","{""1"":""<i data-icon-name=\""CheckMark\"" aria-hid...","{""1"":""as a part of the online application proc..."
724,"<input type=""text"" id=""TextField41"" aria-requi...",2024-07-27T14_35_55_871Z.html,488,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":""data privacy notice"",""1"":""thirdparty coo...",most recent job title,"<input aria-describedby=""TextFieldDescription4...","{""2"":""<span class=\""ms-Button-label label-76\""...","{""2"":""save and continue"",""3"":""add resumeupload..."
725,"<span class=""readOnlyPlaceholder-475"" nid=""511...",2024-07-27T14_36_00_556Z.html,511,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":""data privacy notice"",""1"":""thirdparty coo...",select a month and year,"<span class=""readOnlyPlaceholder-475"" clickabl...","{""3"":""<div class=\""profileResumeContentStyle-3...","{""3"":""add resumeupload a file or generate a re..."


In [5]:
import re
def remove_invalid_unicode_sequences(json_str):
    # Regex to find sequences like \u00f or \u00e4 which are invalid
    pattern = re.compile(r'\\u(?![0-9a-fA-F]{4})[0-9a-fA-F]{0,3}')    
    # Substitute all invalid sequences with an empty string
    cleaned_json_str = re.sub(pattern, '', str(json_str))
    return cleaned_json_str

mypd = mypd.fillna('').map(remove_invalid_unicode_sequences)

In [6]:
train_data = mypd[16:].reset_index(drop=True)
test_data = mypd[:16].reset_index(drop=True)
test_data

,outerHTML,filename,nid,negatives,negatives_query,query,element,history,history_query
0,"<a xml:lang=""en-US"" lang=""en-US"" class=""btn bt...",2024-07-27T06_52_57_411Z.html,434,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""language"",""2"":""sign in"",""3"":""empl...",apply now,"<a class=""btn btn-primary btn-large btn-lg app...",{},{}
1,"<input type=""text"" name=""username"" id=""usernam...",2024-07-27T06_53_08_168Z.html,489,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""5""...",email address,"<input aria-describedby=""signIn_EmailError"" ar...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now""}"
2,"<input type=""password"" name=""password"" id=""pas...",2024-07-27T06_53_10_623Z.html,497,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",password,"<input aria-describedby=""signIn_PasswordError""...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now"",""1"":""email address""}"
3,"<button onclick=""return validateFields();"" nid...",2024-07-27T06_53_12_219Z.html,505,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",sign in register,"<button clickable=""true"" nid=""505"" onclick=""re...","{""0"":""<a xml:lang=\""en-US\"" lang=\""en-US\"" cla...","{""0"":""apply now"",""1"":""email address"",""2"":""pass..."
4,"<a href=""/career?company=Koerber&amp;site=VjIt...",2024-07-27T06_53_21_295Z.html,523,"{""0"":""<a clickable=\""true\"" href=\""https:\/\/k...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",create an account,"<a clickable=""true"" href=""/career?company=Koer...","{""1"":""<input type=\""text\"" name=\""username\"" i...","{""1"":""email address"",""2"":""password"",""3"":""sign ..."
5,"<input class=""form-control"" type=""text"" size=""...",2024-07-27T06_53_28_432Z.html,536,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",email address,"<input aria-describedby=""fbclc_userName_error""...","{""2"":""<input type=\""password\"" name=\""password...","{""2"":""password"",""3"":""sign in register"",""4"":""c..."
6,"<input class=""form-control"" type=""text"" size=""...",2024-07-27T06_53_30_073Z.html,544,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":"""",""4""...",retype email address,"<input aria-describedby=""fbclc_emailConf_error...","{""3"":""<button onclick=\""return validateFields(...","{""3"":""sign in register"",""4"":""create an accoun..."
7,"<input class=""form-control"" id=""fbclc_pwd"" typ...",2024-07-27T06_53_32_058Z.html,554,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":""pleas...",choose password,"<input aria-describedby=""fbclc_pwd_error rcmPw...","{""4"":""<a href=\""\/career?company=Koerber&amp;s...","{""4"":""create an account"",""5"":""email address"",""..."
8,"<input class=""form-control"" type=""password"" si...",2024-07-27T06_53_34_199Z.html,575,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":""pleas...",retype password,"<input aria-describedby=""fbclc_pwdConf_error p...","{""5"":""<input class=\""form-control\"" type=\""tex...","{""5"":""email address"",""6"":""retype email address..."
9,"<input class=""form-control"" type=""text"" size=""...",2024-07-27T06_53_36_160Z.html,586,"{""0"":""<a clickable=\""false\"" href=\""https:\/\/...","{""0"":"""",""1"":""employee login"",""2"":"""",""3"":""pleas...",first name,"<input aria-describedby=""fbclc_fName_error"" ar...","{""6"":""<input class=\""form-control\"" type=\""tex...","{""6"":""retype email address"",""7"":""choose passwo..."


In [7]:
def fix(x:str):
    i=x.rfind('",')
    if x[-1]=='}':
        return x
        pass
    #elif x[-1]=='\\':
    #    x=x+'""}'
    else:
        #x=x+'"}'"""
        return x[:i]+'"}'
    
train_data['negatives_query'] = train_data['negatives_query'].map(fix)
train_data['negatives'] = train_data['negatives'].map(fix)


In [8]:
"""
def get_pair_label(data):
    train_dataset = []
    x_previous=''
    for i,x in data.iterrows():
        #print(i,x['filename'])
        
        hq = list(json.loads(x['history_query']).values())
        he = list(json.loads(x['history']).values())
        
        if len(he)>0:
            hq = str(hq[-1])
            he = he[-1].replace('\n','').replace('  ','')
            
            
        else:
            hq= ''
            he= ''
        
        try:
            nq = json.loads(x['negatives_query'])
        except Exception as e:
            
            print(i,e)
            
        try:
            ne = json.loads(x['negatives'])
        except Exception as e:
            
            print(i,e)

        for q,e in zip(nq.values(),ne.values()):
            train_dataset.append(InputExample(texts=[hq+' '+he,'\n'+q+' '+e],label=0.0))
            
            pass
            
        
        train_dataset.append(InputExample(texts=[hq+' '+he,'\n'+x['query']+' '+x['element']],label=1.0))
        x_previous =x
        #break
    return train_dataset

"""

"\ndef get_pair_label(data):\n    train_dataset = []\n    x_previous=''\n    for i,x in data.iterrows():\n        #print(i,x['filename'])\n        \n        hq = list(json.loads(x['history_query']).values())\n        he = list(json.loads(x['history']).values())\n        \n        if len(he)>0:\n            hq = str(hq[-1])\n            he = he[-1].replace('\n','').replace('  ','')\n            \n            \n        else:\n            hq= ''\n            he= ''\n        \n        try:\n            nq = json.loads(x['negatives_query'])\n        except Exception as e:\n            \n            print(i,e)\n            \n        try:\n            ne = json.loads(x['negatives'])\n        except Exception as e:\n            \n            print(i,e)\n\n        for q,e in zip(nq.values(),ne.values()):\n            train_dataset.append(InputExample(texts=[hq+' '+he,'\n'+q+' '+e],label=0.0))\n            \n            pass\n            \n        \n        train_dataset.append(InputExample(text

In [9]:
import datasets
from sentence_transformers import InputExample
import random
def pair(q,e):
    i = random.randint(0,2)

    if i==0:
        return q
    elif i==1:
        return e
    elif i==2 :
        return q+' '+e
    
def get_pair_label(data):
    train_dataset = []
    x_previous=''
    for i,x in data.iterrows():
        #print(i,x['filename'])
        
        hq = list(json.loads(x['history_query']).values())
        he = list(json.loads(x['history']).values())
        
        if len(he)>0:
            hq = str(hq[-1])
            he = he[-1].replace('\n','').replace('  ','')
            
            
        else:
            hq= ''
            he= ''
            
        H = pair(hq,he)
        
        try:
            nq = json.loads(x['negatives_query'])
        except Exception as e:
            
            print(i,e)
            
        try:
            ne = json.loads(x['negatives'])
        except Exception as e:
            
            print(i,e)
        
        pq = ''
        pe = ''
        for q,e in zip(nq.values(),ne.values()):
            #p1 = '{'+pair(pq,pe)+'}'
            p2 = pair(q,e)
            train_dataset.append(InputExample(texts=[H,'\n'+p2],label=0.0))
            pe = e
            pq = q
            
            pass
        
          
        p1 = pair(x['query'],x['element'])
        train_dataset.append(InputExample(texts=[H,'\n'+p1],label=1.0))
        x_previous =x
        #break
    return train_dataset

#test_dataset = []
#for i,x in test_data.iterrows():
#    test_dataset.append([x['history_query'],x['query']])
train_dataset = get_pair_label(train_data)
print(train_dataset[151].texts)


['upload from device <input tabindex="0" type="file" aria-labelledby="48:_uploadComputer" id="48:_file" class="fileUpload" name="fileData1" onchange="juic.fire(&quot;48:&quot;,&quot;uploadFiles&quot;,event);" onfocus="juic.fire(&quot;48:&quot;,&quot;_focusOnLabel&quot;,event);" onblur="juic.fire(&quot;48:&quot;,&quot;_focusOutOfLabel&quot;,event);" onkeydown="juic.fire(&quot;48:&quot;,&quot;onKeydown&quot;,event);" ni', '\nexpand all sections']


In [10]:
for x in train_dataset:
    if x.label==1.0:
        print(x.texts)

['', '\n<a class="btn btn-primary btn-large btn-lg apply dialogApplyBtn" clickable="true" href="/talentcommunity/apply/734172001/?locale=en_US" lang="en-US" nid="434" visible="true" xml:lang="en-US">Apply now »</a>']
['apply now <a xml:lang="en-US" lang="en-US" class="btn btn-primary btn-large btn-lg apply dialogApplyBtn " href="/talentcommunity/apply/734172001/?locale=en_US" nid="434" clickable="true" visible="true">Apply now »</a>', '\n<input aria-describedby="signIn_EmailError" aria-invalid="false" aria-required="true" class="" clickable="false" id="username" maxlength="99" name="username" nid="489" onblur="validateEmail(event);" onpaste="SFDOMEvent.preventDefault(event);" size="30" type="text" value="" visible="true"/>']
['email address', '\npassword']
['password', '\nsign in  register']
['sign in  register', '\n']
['<span tabindex="0" role="button" aria-labelledby="38: 47:_ariaActionLabel" id="47:_attachIcon" class="glyphicon glyphicon-pencil addAttachments" onclick="juic.fire(&qu

In [11]:


from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset

class customDataset(Dataset):
    def __init__(self,data) -> None:
        super().__init__()
        self.data = data
        self.update()
        
    def __getitem__(self, index) :
        return self.dataset[index]
    def __len__(self):
        return len(self.dataset)
    def update(self):
        print('update')
        self.dataset = get_pair_label(self.data)

cd = customDataset(train_data)
dl = DataLoader(cd,batch_size=32,collate_fn=model.smart_batching_collate)
print(len(dl))
next(iter(dl))


update
1087


({'input_ids': tensor([[    0,     2,     2,  ...,     1,     1,     1],
         [    0,     2,     2,  ...,     1,     1,     1],
         [    0,     2,     2,  ...,     1,     1,     1],
         ...,
         [    0, 59911,  5036,  ...,     1,     1,     1],
         [    0, 59911,  5036,  ...,     1,     1,     1],
         [    0, 59911,  5036,  ...,     1,     1,     1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')},
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        device='cuda:0'))

In [12]:
test_dataset = get_pair_label(test_data)
def myeval(test_dataset):
    docs=[]
    count = 0
    acc=0.0
    for i,t in enumerate(test_dataset):
        docs.append(t.texts[1])
        if t.label==1.0:
            query = t.texts[0]
            r = model.rank(query,docs)
            id = r[0]['corpus_id']+1
           
            if id==len(docs):
                acc+=1
            
            count+=1
            docs=[]
            
    return acc/count
        
#train_acc= myeval(train_dataset)
test_acc= myeval(test_dataset)

def f():
    dl.dataset.update()
    pass

def ef(*a,**ka):
    acc =  myeval(test_dataset)
    print('epoch',ka['epoch'],'Acc:',acc)
    return acc
    

model.fit(
    train_dataloader=dl,
    evaluator=ef,
    epochs=20,
    #warmup_steps=warmup_steps,
    output_path=cache_dir,
    optimizer_params = { "lr": 1e-5 },
    
    train_callback=f

)

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1087 [00:00<?, ?it/s]

epoch 0 Acc: 0.0625
[2024-08-02 22:19:02,811] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
update


Iteration:   0%|          | 0/1087 [00:00<?, ?it/s]

epoch 1 Acc: 0.0625
update


...

epoch 8 Acc: 0.0625
update


Iteration:   0%|          | 0/1087 [00:00<?, ?it/s]

epoch 9 Acc: 0.0625
update


Iteration:   0%|          | 0/1087 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test_acc= myeval(train_dataset)
test_acc

0.9634317862165963

In [ ]:
#model.save_pretrained(output_dir+'click_2')

In [ ]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "This beautiful waves ring is crafted in platinum and is set with blue sapphires and white diamonds."

# Process the text
doc = nlp(text)

# Extract SVO triples
def extract_svo(doc):
    triples = []
    for token in doc:
        if token.dep_ == "nsubj" and token.head.pos_ == "VERB":
            subject = token.text
            verb = token.head.text
            # Find the direct object
            obj = [child.text for child in token.head.children if child.dep_ == "dobj"]
            if obj:
                triples.append((subject, verb, obj[0]))
    return triples

# Extract triples from the document
svo_triples = extract_svo(doc)
print(svo_triples)


[]


In [ ]:
import spacy

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Sample text
text = "Alice gave Bob a book."

# Process the text
doc = nlp(text)

# Extract SVO triples
for token in doc:
    print(token,token.dep_)
    if token.dep_ in ("nsubj", "dobj"):
        subject = token.text if token.dep_ == "nsubj" else None
        verb = token.head.text
        obj = token.text if token.dep_ == "dobj" else None
        if subject and obj:
            print(f"Subject: {subject}, Verb: {verb}, Object: {obj}")


Alice nsubj
gave ROOT
Bob dative
a det
book dobj
. punct
